# Various Conversation Chains and Memory with LCEL

1) ConversationBufferMemory

2) ConversationBufferWindowMemory


3)ConversationSummaryMemory

4)VectorStoreRetrieverMemory


5)ChatMessageHistory


6)SQLChatMessageHistory

In [1]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-community
!pip install langchain-chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.6 MB/s eta 0:00:00


In [2]:
from getpass import getpass
OPENAI_API_KEY=getpass("enter key :-")

enter key :-··········


In [3]:
import os
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [4]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o-mini",temperature=0)

#1) ConversationBufferMemory

In [ ]:
from langchain.memory import ConversationBufferMemory
memory=ConversationBufferMemory(return_messages=True) #The conversation history is stored and returned as a list of AIMessage and HumanMessage objects instead of a single string.

/tmp/ipython-input-3969836440.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory(return_messages=True) #The conversation history is stored and returned as a list of AIMessage and HumanMessage objects instead of a single string.


In [ ]:
memory.load_memory_variables({})

{'history': []}

In [ ]:
def get_history(query):
  return memory.load_memory_variables(query)['history']  #Currently → query is just passed along because LangChain calls all RunnableLambda functions with the current inputs.
                                                          #By default → You get complete history, not filtered.

In [5]:
system_prompt="you are the teacher of 10th standard. Students will interact with you so respond accordingly."

In [6]:
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages([('system',system_prompt),
MessagesPlaceholder(variable_name='history'),
                                         ('human','{query}')
                                         ])

In [ ]:
from langchain.schema.runnable import RunnableLambda,RunnablePassthrough

chain=(   RunnablePassthrough.assign(history=RunnableLambda(get_history))
                      |
                    prompt
                      |
                    llm
)

In [ ]:
def student_chatBot():
  query=input("enter your query:")
  response=chain.invoke({'query':query})
  print(response.content)
  memory.save_context(inputs={'query':query},outputs={'output':response.content})

In [ ]:
student_chatBot()

enter your query:hello mam!
Hello! How are you today? Is there something specific you’d like to discuss or ask about?


In [ ]:
student_chatBot()

enter your query:mam ! my studies not going good , i face lot of difficulties in maths and science subject . what should i do ?
I understand that math and science can be challenging subjects for many students. Here are a few strategies that might help you improve:

1. **Practice Regularly**: Consistent practice is key in math and science. Try to solve a few problems every day to build your confidence.

2. **Understand the Concepts**: Instead of just memorizing formulas, focus on understanding the underlying concepts. This will help you apply what you learn to different problems.

3. **Ask Questions**: Don’t hesitate to ask questions in class or during study sessions. If something isn’t clear, it’s important to seek clarification.

4. **Study Groups**: Consider forming a study group with classmates. Explaining concepts to each other can reinforce your understanding.

5. **Use Online Resources**: There are many educational websites and videos that can provide additional explanations and 

In [ ]:
student_chatBot()

enter your query:which subject can you tech me better in above mention ?
I can help you with both math and science! If you have a specific topic or concept in either subject that you're struggling with, let me know, and we can work through it together. Whether it's algebra, geometry, physics, or chemistry, I'm here to assist you. Which subject would you like to focus on first?


In [ ]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hello mam ! how are you ?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hello! I'm doing well, thank you for asking. How about you? How's everything going with your studies?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='mam ! my studies not going good , i face lot of difficulties in maths and science subject . what should i do ?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I understand that math and science can be challenging subjects for many students. Here are a few strategies that might help you improve:\n\n1. **Practice Regularly**: Consistent practice is key in math and science. Try to solve a few problems every day to build your confidence.\n\n2. **Understand the Concepts**: Instead of just memorizing formulas, focus on understanding the underlying concepts. This will help you apply what you learn to different problems.\n\n3. **Ask Questions**: Don’t hesitate to ask questions 

# 2) ConversationBufferWindowMemory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
memory=ConversationBufferWindowMemory(return_messages=True,k=3)

In [ ]:
student_chatBot()

enter your query:may come in , mam ?
Of course! Please come in. How can I assist you today?


In [ ]:
student_chatBot()

enter your query:mam, i got less marks in maths , i am fearing that i will be fail in upcoming half yearly examnation !
I understand how you feel, and it's completely normal to be worried about your performance. The important thing is to focus on what you can do to improve. Let's take a look at a few steps you can take:

1. **Identify Weak Areas**: Try to figure out which topics you find most challenging. This will help you focus your study efforts.

2. **Practice Regularly**: Mathematics requires practice. Set aside some time each day to work on problems, especially in the areas where you struggle.

3. **Ask for Help**: Don’t hesitate to ask questions in class or seek help from me or your classmates. Sometimes, a different explanation can make things clearer.

4. **Use Resources**: There are many online resources, videos, and practice worksheets that can help you understand concepts better.

5. **Stay Positive**: Believe in yourself! A positive mindset can make a big difference in you

In [ ]:
student_chatBot()

enter your query:mam ,  i remmeber all formulas very well but in examination room i forgot all i remmber !
It's quite common to forget formulas or concepts during an exam, especially if you're feeling anxious. Here are some strategies to help you remember them better:

1. **Practice Under Exam Conditions**: Try to simulate exam conditions while studying. Set a timer and practice solving problems without looking at your notes. This can help you get used to recalling information under pressure.

2. **Create a Formula Sheet**: Write down all the important formulas on a sheet of paper. Review it regularly, and try to recall the formulas without looking. This can reinforce your memory.

3. **Understand, Don’t Just Memorize**: Make sure you understand the concepts behind the formulas. When you understand how and why a formula works, it’s easier to remember it.

4. **Use Mnemonics**: Create acronyms or phrases to help you remember sequences or groups of formulas. This can make recall easier d

In [ ]:
student_chatBot()

enter your query:mam , should i take toution other than school classes ?
Whether or not to take tuition classes depends on your individual needs and learning style. Here are some factors to consider:

1. **Understanding of Concepts**: If you feel that you’re struggling to grasp certain concepts in class, additional help from a tutor can provide you with more personalized attention and explanations.

2. **Learning Style**: Some students benefit from one-on-one instruction, while others may do well with self-study or group study. Think about what works best for you.

3. **Time Management**: Consider your schedule. If you have the time to commit to tuition without feeling overwhelmed, it could be beneficial. However, if it adds too much stress, it might be better to focus on self-study or seek help from your teacher.

4. **Cost**: Tuition can be an additional expense. Make sure it fits within your budget and that you feel it will be worth the investment.

5. **School Resources**: Don’t fo

In [ ]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='mam, i got less marks in maths , i am fearing that i will be fail in upcoming half yearly examnation !', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I understand how you feel, and it's completely normal to be worried about your performance. The important thing is to focus on what you can do to improve. Let's take a look at a few steps you can take:\n\n1. **Identify Weak Areas**: Try to figure out which topics you find most challenging. This will help you focus your study efforts.\n\n2. **Practice Regularly**: Mathematics requires practice. Set aside some time each day to work on problems, especially in the areas where you struggle.\n\n3. **Ask for Help**: Don’t hesitate to ask questions in class or seek help from me or your classmates. Sometimes, a different explanation can make things clearer.\n\n4. **Use Resources**: There are many online resources, videos, and practice worksheets that can help you understand concepts better.\n

we can see clearly here , that only previous 3 records store in memory , older ones are deleted automatically.


# 3) ConversationSummaryMemory

In [ ]:
from langchain.memory import ConversationSummaryMemory
memory=ConversationSummaryMemory(return_messages=True,llm=llm)

/tmp/ipython-input-3332318420.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationSummaryMemory(return_messages=True,llm=llm)


In [ ]:
student_chatBot()

enter your query:may i come in mam ?
Of course! Please come in. How can I assist you today?


In [ ]:
student_chatBot()

enter your query:mam , my parents can't afford my huge fees , i can't pay this month of fee because my father get huge loss in share market . what should i do ?
I’m really sorry to hear about your situation. It’s important to communicate openly with your school about your financial difficulties. Here are a few steps you can take:

1. **Talk to Your Teacher or Counselor**: Explain your situation to a trusted teacher or school counselor. They may be able to help you find a solution or provide guidance on what to do next.

2. **Payment Plans**: Ask if the school offers any payment plans or financial assistance programs. Many schools have options for students facing financial hardships.

3. **Scholarships or Grants**: Look into any scholarships or grants that you might be eligible for. Some organizations offer financial support to students in need.

4. **Temporary Leave**: If necessary, discuss the possibility of a temporary leave from school until your family’s financial situation improve

In [ ]:
student_chatBot()

enter your query:mam , because tention , my marks also going low and now school adminstration warn me to fire from school if i won't get 70% marks in maths and science .what should i do ?
I understand that you're feeling a lot of pressure right now, and it's completely normal to feel overwhelmed in such situations. Here are some steps you can take to improve your situation:

1. **Talk to Your Teachers**: Reach out to your math and science teachers. Let them know about your difficulties. They may be able to provide extra help or resources.

2. **Create a Study Plan**: Organize your study time. Break down your subjects into manageable sections and set specific goals for each study session.

3. **Join Study Groups**: Sometimes studying with classmates can help you understand concepts better. You can learn from each other and motivate one another.

4. **Practice Regularly**: Make sure to practice problems regularly, especially in math. The more you practice, the more confident you will bec

In [ ]:
memory.load_memory_variables({})

{'history': [SystemMessage(content="The human expresses concern about their declining marks in math and science, mentioning that the school administration has warned them about potential expulsion if they don't achieve 70% in those subjects. The AI acknowledges the pressure the human is feeling and suggests several steps to improve their situation: talking to teachers for extra help, creating a study plan, joining study groups, practicing regularly, maintaining a positive mindset, taking care of their health, and seeking additional resources. The AI reassures the human that it's okay to ask for help and emphasizes the importance of making small improvements.", additional_kwargs={}, response_metadata={})]}

__we can see how previous conversation saved as summary form as system message.__

# 4)VectorStoreRetrieverMemory

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings


In [ ]:
embaddor=OpenAIEmbeddings(model='text-embedding-3-small')

In [ ]:
db=Chroma(collection_name='history',embedding_function=embaddor)

In [ ]:
retrival=db.as_retriever(search_type='similarity',search_kwargs={'k':4})

In [ ]:
from langchain.memory import VectorStoreRetrieverMemory


In [ ]:
memory=VectorStoreRetrieverMemory(return_messages=True,retriever=retrival)

/tmp/ipython-input-2190742186.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=VectorStoreRetrieverMemory(return_messages=True,retriever=retrival)


In [ ]:
memory.load_memory_variables({'query':'hii'})

{'history': ''}

__we can noticed that history key have value not in list , it's in string format .__ so we need to modify our get_history method .

In [ ]:
def get_history_from_vetorStore(query):
  return [get_history(query=query)]

In [ ]:
def vector_studentchatBot():
  query=input("enter query:")
  vector_chain=(
      RunnablePassthrough.assign(history=RunnableLambda(get_history_from_vetorStore))
              |
            prompt
              |
              llm
  )
  response=vector_chain.invoke({'query':query})
  memory.save_context({'query':query},{'output':response.content})
  print(response.content)

In [ ]:
vector_studentchatBot()

enter query:hello mam ! my self govinda , your 10th standard student .
Hello Govinda! It's great to meet you. How are you doing today? Is there anything specific you'd like to discuss or any questions you have about your studies?


In [ ]:
vector_studentchatBot()

enter query:i am good mam, actually i failed in maths subject  in 2nd test.
I'm sorry to hear that, Govinda. It's okay to face challenges sometimes. The important thing is to learn from them and improve. Would you like to talk about what specific topics you found difficult in math? We can work on a plan to help you understand better and prepare for the next test.


In [ ]:
vector_studentchatBot()

enter query:mam, can you give me tip for paasing out in this subject ?
Of course, Govinda! Here are some tips to help you pass in math:

1. **Understand the Basics**: Make sure you have a strong grasp of the fundamental concepts. If there are any topics you find confusing, let me know, and we can go over them together.

2. **Practice Regularly**: Math requires practice. Try to solve a variety of problems every day. This will help reinforce your understanding and improve your problem-solving skills.

3. **Use Study Resources**: Utilize textbooks, online resources, and videos that explain concepts in different ways. Sometimes a different explanation can make things click.

4. **Form a Study Group**: Studying with classmates can be very helpful. You can help each other with difficult topics and share different methods of solving problems.

5. **Ask Questions**: Don’t hesitate to ask questions in class or during study sessions. If something isn’t clear, it’s important to seek clarification

# 5) ChatMessageHistory

In [7]:
new_chain=prompt | llm

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

history={}
def get_message_history(session_id:str):
  if session_id not in history:
    history[session_id]=ChatMessageHistory()
  return history[session_id]

main_chain=RunnableWithMessageHistory(new_chain,get_session_history=get_message_history,input_messages_key='query',history_messages_key='history')

def student_chatBot():
  query=input("enter your query:")
  session_id=input("enter your session_id:")
  response=main_chain.invoke({'query':query}, {'configurable': { 'session_id': session_id}})
  print(response.content)


In [10]:
student_chatBot()

enter your query:hello mam , govinda this side 
enter your session_id:ghk
Hello Govinda! How are you today? Is there something specific you would like to discuss or any questions you have about your studies?


In [11]:
student_chatBot()

enter your query:mam ! u know my name ?
enter your session_id:ghk
Yes, you just introduced yourself as Govinda! How can I assist you today?


#6)SQLChatMessageHistory

In [20]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables import RunnablePassthrough

def get_message_history(session_id:str):
  return SQLChatMessageHistory(session_id,"sqlite:///memory.db")

def filter_history(messages,k=3):
  return messages[-k:]

new_chain=(
    RunnablePassthrough.assign(history=lambda x:filter_history(x["history"]))
    |
    prompt
    |
    llm
)
main_chain=RunnableWithMessageHistory(new_chain,get_session_history=get_message_history,input_messages_key='query',history_messages_key='history')
def student_chatBot():
  query=input("enter your query:")
  session_id=input("enter your session_id:")
  response=main_chain.invoke({'query':query}, {'configurable': { 'session_id': session_id}})
  print(response.content)


In [21]:
student_chatBot()

enter your query:hello mam , do you know my name ?
enter your session_id:gh0
Yes, I do! You just introduced yourself as Govinda. How can I assist you today?


In [22]:
student_chatBot()

enter your query:mam ! which subject u can tech me ?
enter your session_id:gk8
I can help you with a variety of subjects, including Mathematics, Science, English, Social Studies, and more. If you have a specific subject or topic in mind that you need help with, just let me know!
